In [3]:
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator

c:\Python39\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
import pandas as pd
import torch
from torch.utils.data import DataLoader
from transformers import BertTokenizer, BertModel
import torch.nn.functional as F
from sentence_transformers import util, losses
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample
from sentence_transformers.evaluation import EmbeddingSimilarityEvaluator
import xgboost as xgb
from sklearn.metrics import confusion_matrix
from sklearn.metrics import classification_report

In [1]:
fine_tuning_file_dir = './data/computers_train/computers_train_large.json.gz'
ft_valid_file_dir = './data/computers_valid/computers_valid_large.csv'
n_epochs = 1

In [7]:
# finetuning bert model
large_df = pd.read_json(fine_tuning_file_dir, compression='gzip', lines=True)
large_df = large_df.drop(columns=['id_left', 'id_right', 'cluster_id_left', 'cluster_id_right', 'identifiers_left', 'identifiers_right'])
valid_ids = pd.read_csv(ft_valid_file_dir)
ft_train_df = large_df.loc[~large_df['pair_id'].isin(valid_ids['pair_id'])]

In [8]:

titles_left = list(ft_train_df['title_left'])
titles_right = list(ft_train_df['title_right'])

descriptions_left = list(ft_train_df['description_left'])
descriptions_right = list(ft_train_df['description_right'])


In [9]:
train_samples = []
for title_left, title_right in zip(titles_left, titles_right):
    train_samples.append(InputExample(
        texts=[title_left, title_right]
    ))

train_dataloader = DataLoader(
    train_samples, batch_size=8
)

# title_model = SentenceTransformer("sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2")
# loss = losses.MultipleNegativesRankingLoss(title_model)
# warmup_steps = int(len(train_dataloader) * n_epochs * 0.1)
# evaluator = EmbeddingSimilarityEvaluator.from_input_examples(train_samples, name='sts-dev')

In [10]:
title_model.fit(
    train_objectives=[(train_dataloader, loss)],
    evaluator=evaluator,
    epochs=n_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path='./title_model_finetuned')

NameError: name 'title_model' is not defined

In [19]:
train_samples = []
for desc_left, desc_right in zip(descriptions_left, descriptions_right):
    if desc_left != None and desc_right != None:
        train_samples.append(InputExample(
            texts=[desc_left, desc_right]
        ))

train_dataloader = DataLoader(
    train_samples, batch_size=8
)

desc_model = SentenceTransformer("sentence-transformers/all-MiniLM-L12-v2")
loss = losses.MultipleNegativesRankingLoss(desc_model)
warmup_steps = int(len(train_dataloader) * n_epochs * 0.1)
evaluator = EmbeddingSimilarityEvaluator.from_input_examples(train_samples, name='sts-dev')

In [40]:
desc_model.fit(
    train_objectives=[(train_dataloader, loss)],
    evaluator=evaluator,
    epochs=n_epochs,
    evaluation_steps=1000,
    warmup_steps=warmup_steps,
    output_path='./desc_model'
    )

Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/48 [00:00<?, ?it/s]

/opt/conda/lib/python3.7/site-packages/scipy/stats/stats.py:3845: PearsonRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(PearsonRConstantInputWarning())
/opt/conda/lib/python3.7/site-packages/scipy/stats/stats.py:4196: SpearmanRConstantInputWarning: An input array is constant; the correlation coefficent is not defined.
  warnings.warn(SpearmanRConstantInputWarning())


In [41]:
sbert_model_mult = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')
sbert_model_eng = SentenceTransformer('sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2')

In [42]:
def get_similarity(model, s_pair):
    if s_pair[0] == None or s_pair[1] == None:
        return None 

    if s_pair[0] == s_pair[1]:
        return 1.0
    
    embds = model.encode(s_pair)
    sim_score = util.cos_sim(embds[0], embds[1]).item()
    return sim_score

def get_similarity_keyval(dic1, dic2):
    key_set = set()
    tot_sim = 0
    for key, value1 in zip(dic1.keys(), dic1.values()):
        if key in dic2:
            value2 = dic2[key]
            if value2 == value1:
                tot_sim += 1

        key_set.add(key)

    for key2 in dic2:
        key_set.add(key2)

    return tot_sim / len(key_set)

In [45]:
data_df = pd.read_json('../data/computers_train/computers_train_medium.json.gz', compression='gzip', lines=True)
data_df = data_df.drop(columns=['id_left', 'id_right', 'cluster_id_left', 'cluster_id_right', 'identifiers_left', 'identifiers_right'])

valid_ids = pd.read_csv('../data/computers_valid/computers_valid_medium.csv')

train_df = data_df.loc[~data_df['pair_id'].isin(valid_ids['pair_id'])]
valid_df = data_df.loc[data_df['pair_id'].isin(valid_ids['pair_id'])]

In [47]:
def compute_features(df):
    df['title_sim'] = df.apply(lambda x: get_similarity(title_model, (x['title_left'], x['title_right'])), axis=1)
    print("title done")
    df['description_sim'] = df.apply(lambda x: get_similarity(desc_model, (x['description_left'], x['description_right'])), axis=1)
    print("description done")
    df['brand_sim'] = df.apply(lambda x: get_similarity(sbert_model_eng, (x['brand_left'], x['brand_right'])), axis=1)
    print("brand done")
    df['cat_sim'] = df.apply(lambda x: get_similarity(sbert_model_eng, (x['category_left'], x['category_right'])), axis=1)
    print("cat done")
    df = df.drop(columns=['title_left', 'description_left', 'brand_left', 'price_left', 'specTableContent_left', \
    'category_left', 'title_right', 'description_right', 'brand_right', 'price_right', 'specTableContent_right',
    'category_right', 'pair_id'
    ])
    return df

In [49]:
num_train_df = compute_features(train_df)

/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


title done


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  after removing the cwd from sys.path.


description done


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


brand done
cat done


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  


In [ ]:
num_valid_df = compute_features(valid_df)

In [58]:
#importing dataset
#isolate the x and y variables
y_train = np.array(num_train_df['label'])
X_train = num_train_df.drop(columns=["label"])
    
y_test = np.array(num_valid_df['label'])
X_test = num_valid_df.drop(columns=["label"])


In [ ]:

#create xgboost matrices
Train = xgb.DMatrix(X_train, label = y_train)
Test = xgb.DMatrix(X_test, label = y_test)

#set the parameters
parameters = {'learning_rate': 0.3,
               'max_depth': 2,
               'colsample_bytree': 1,
               'subsample': 1,
               'min_child_weight': 1,
               'gamma': 0, 
               'random_state': 1500,
               'eval_metric': "auc",
               'objective': "binary:logistic"}


model = xgb.train(params = parameters,
                dtrain = Train,
                num_boost_round = 200,
                evals = [(Test, "Yes")],
                verbose_eval = 50)


#PRedictions
pred = model.predict(Test)
pred = np.where(pred > 0.5, 1, 0)

confusion_matrix = confusion_matrix(y_test, pred)
report = classification_report(y_test, pred)
print(report)
